This notebook demonstrates SQL UDFs in Databricks for inventory and temperature management:

- **Supplier Lookup:** Uses `get_supplier_details` to fetch supplier info.
- **Backup Inventory:** Defines `get_backup_inventory` to list available stock by product ID, ordered by expiry.
- **Function Usage:** Shows how to query backup inventory for a product.
- **Temperature Gap:** Implements `temp_gap` to calculate the difference between ambient and safe product temperatures.

In [0]:
%pip install -U -qqqq unitycatalog-ai[databricks] mlflow-skinny[databricks] langgraph==0.3.4 databricks-langchain databricks-agents python-dotenv uv
%restart_python

In [0]:
import sys
sys.path.append(".")

In [0]:
dbutils.widgets.removeAll()

In [0]:

# Create widgets in order to bind environment variables to the SQL session variables so they can be used in %sql cells
dbutils.widgets.text("TARGET_CATALOG", "workshop")
dbutils.widgets.text("TARGET_SCHEMA", "supply_chain")

TARGET_CATALOG = dbutils.widgets.get("TARGET_CATALOG")
TARGET_SCHEMA = dbutils.widgets.get("TARGET_SCHEMA")


print(f"TARGET_CATALOG: {TARGET_CATALOG}")
print(f"TARGET_SCHEMA: {TARGET_SCHEMA}")

In [0]:
%sql
USE IDENTIFIER(:TARGET_CATALOG || "." || :TARGET_SCHEMA);

In [0]:
%sql
CREATE OR REPLACE FUNCTION get_shipments(
  dest_city STRING COMMENT 'Optional: City or region to filter (e.g. NYC)',
  shipment_status STRING COMMENT 'Optional: Filter by shipment status (e.g. in-transit, delivered, delayed). Use NULL for all.'
)
RETURNS TABLE(
  shipment_id STRING COMMENT 'Shipment ID',
  product_id STRING COMMENT 'Product ID',
  product_name STRING COMMENT 'Product name',
  supplier_id STRING COMMENT 'Supplier ID',
  destination STRING COMMENT 'Destination city',
  eta_date DATE COMMENT 'ETA delivery date',
  status STRING COMMENT 'Shipment status',
  temperature_max_f DOUBLE COMMENT 'Max safe temp (°F)'
)
LANGUAGE SQL
COMMENT 'Retrieve shipments optionally filtered by city and/or status.'
RETURN
(
  SELECT
    shipment_id,
    product_id,
    product_name,
    supplier_id,
    destination,
    CAST(eta_date AS DATE) AS eta_date,
    status,
    temperature_max_f
  FROM shipments
  WHERE
    -- Filter by destination if provided
    (
      dest_city IS NULL
      OR LOWER(destination) LIKE CONCAT('%', LOWER(dest_city), '%')
      OR (
        LOWER(dest_city) IN ('ny', 'nyc', 'new york', 'new york city')
        AND LOWER(destination) LIKE '%new york%'
      )
    )
    -- Filter by status if provided
    AND (
      shipment_status IS NULL
      OR LOWER(status) = LOWER(shipment_status)
    )
  ORDER BY eta_date ASC
);

In [0]:
%sql
SELECT * FROM get_shipments('New York', 'in-Transit');

In [0]:
%sql
CREATE OR REPLACE FUNCTION get_supplier_details(
  supplier_name_in STRING DEFAULT NULL COMMENT 'Supplier name (case-insensitive, optional)',
  supplier_id_in   STRING DEFAULT NULL COMMENT 'Supplier ID (optional)'
)
RETURNS TABLE (
  supplier_id   STRING COMMENT 'Supplier ID',
  supplier_name STRING COMMENT 'Supplier name',
  contact_name  STRING COMMENT 'Main contact name',
  contact_email STRING COMMENT 'Contact email',
  phone         STRING COMMENT 'Contact phone',
  tier          STRING COMMENT 'Supplier tier level'
)
LANGUAGE SQL
COMMENT 'Gets supplier contact and escalation info by supplier name or ID. If only one argument is passed, the other defaults to NULL.'
RETURN
(
  WITH params AS (
    SELECT
      NULLIF(supplier_id_in, '')   AS sid,
      NULLIF(supplier_name_in, '') AS sname
  )
  SELECT 
      s.supplier_id,
      s.supplier_name,
      s.contact_name,
      s.contact_email,
      s.phone,
      s.tier
  FROM suppliers s, params p
  WHERE
        (p.sid IS NOT NULL AND s.supplier_id = p.sid)
     OR (p.sid IS NULL AND p.sname IS NOT NULL 
         AND LOWER(s.supplier_name) LIKE LOWER(CONCAT('%', p.sname, '%')))
  LIMIT 1
);

In [0]:
%sql
SELECT * FROM get_supplier_details('MedAxis')
UNION ALL
SELECT * FROM get_supplier_details('', 'SUP-010');

In [0]:
%sql
CREATE OR REPLACE FUNCTION get_backup_inventory(
  product_id STRING COMMENT 'Product ID or partial ID'
)
RETURNS TABLE(
  site_id STRING COMMENT 'Warehouse ID',
  site_name STRING COMMENT 'Warehouse name',
  city STRING COMMENT 'Warehouse city',
  state STRING COMMENT 'Warehouse state',
  product_id STRING COMMENT 'Product ID',
  product_name STRING COMMENT 'Product name',
  supplier_id STRING COMMENT 'Supplier ID',
  on_hand_qty INT COMMENT 'Available quantity',
  lot_expiry DATE COMMENT 'Lot expiry date'
)
LANGUAGE SQL
COMMENT 'Returns backup stock by product ID.'
RETURN
(
  SELECT
    site_id,
    site_name,
    city,
    state,
    product_id,
    product_name,
    supplier_id,
    on_hand_qty,
    CAST(lot_expiry AS DATE) AS lot_expiry
  FROM inventory
  WHERE LOWER(inventory.product_id) LIKE LOWER(CONCAT('%', product_id, '%'))
    AND on_hand_qty > 0
  ORDER BY lot_expiry ASC
);

In [0]:
%sql
SELECT * FROM get_backup_inventory('PRD-001');

In [0]:
%sql
-- One function: tool-friendly + SQL-friendly
CREATE OR REPLACE FUNCTION temp_gap(
  weather_f_str STRING DEFAULT NULL COMMENT 'Ambient temp in °F (e.g., 70 or 70.2). String to avoid tool-schema regex.',
  max_temp_f_str STRING DEFAULT NULL COMMENT 'Max allowable temp in °F (string).'
)
RETURNS DOUBLE
LANGUAGE SQL
COMMENT 'Absolute temp gap (°F). Accepts strings, casts internally; returns NULL on invalid input.'
RETURN (
  WITH c AS (
    SELECT
      TRY_CAST(NULLIF(weather_f_str, '')  AS DECIMAL(10,2)) AS wf,
      TRY_CAST(NULLIF(max_temp_f_str, '') AS DECIMAL(10,2)) AS mf
  )
  SELECT
    CASE
      WHEN wf IS NULL OR mf IS NULL THEN NULL
      ELSE ROUND(ABS(mf - wf), 2)
    END
  FROM c
);

In [0]:
%sql
SELECT
  shipment_id,
  temperature_max_f,
  temp_gap(50, temperature_max_f) AS diff
FROM shipments;